In [1]:
import pandas as pd
import numpy as np
import plotly as plt
import plotly.express as px
import io

## Reading in assessment data for every TN Public school for 5 years of historical data (technically 6 but the test was cancled for 2020)

In [2]:
# Missing data related to district, school, and enrollment 
data_2017 = pd.read_csv('../data/school_based/2017.csv')
# Missing enrollment data
data_2018 =pd.read_csv('../data/school_based/2018.csv')
data_2019 = pd.read_csv('../data/school_based/2019.csv')
data_2021 = pd.read_csv('../data/school_based/2021.csv')
# Uses different naming conventions than previous years.
# Metric naming will be standardized to align with 2022 conventions
data_2022 = pd.read_csv('../data/school_based/2022.csv')

In [3]:
# List of annual assessment daframes
dfs = [data_2017, data_2018, data_2019, data_2021, data_2022]
df_names = ['year_2017', 'year_2018', 'year_2019', 'year_2021', 'year_2022']

# Initialize a StringIO object to get info in one go
output = io.StringIO()

# Loop through the dfs and their names
for df, name in zip(dfs, df_names):

    # Write the name of the DataFrame to the output
    output.write(f"{name} info:\n")
    
    # Capture the output of the .info() method
    df.info(buf=output)
    
    # Separater
    output.write("\n---\n")

# output 
info_str = output.getvalue()

# Close the StringIO object
output.close()

print(info_str)

year_2017 info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440633 entries, 0 to 440632
Data columns (total 16 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   year             440633 non-null  int64  
 1   system           440633 non-null  int64  
 2   school           440633 non-null  int64  
 3   subject          440633 non-null  object 
 4   grade            440633 non-null  object 
 5   subgroup         440633 non-null  object 
 6   valid_tests      440633 non-null  float64
 7   n_below          440633 non-null  object 
 8   n_approaching    440633 non-null  object 
 9   n_on_track       440633 non-null  object 
 10  n_mastered       440633 non-null  object 
 11  pct_below        440633 non-null  object 
 12  pct_approaching  440633 non-null  object 
 13  pct_on_track     440633 non-null  object 
 14  pct_mastered     440633 non-null  object 
 15  pct_on_mastered  440633 non-null  object 
dtypes: float64(1), int64(3

## Issues
- 2017 does not have the `school_name`, `system_name`, and `test` columns.  
  - I will create a mapping from the merged dataframes for subsequent years to backfill the missing strings.
- Figure out to deal with suppressed data and salvage as much student demographic data as you can      
  - `*` = The number of **valid** test scores is less than 10.
  - `**` = Any individual proficiency level is <u>less than</u> **1%** or <u>greater than</u> **99%** of the district average.


### Combine Datasets with more or less the same naming conventions

In [4]:
middle_years = pd.concat([data_2018, data_2019, data_2021])

# Renaming columns in middle years to match 2022 naming conventions.  They mean the exact same thing.
middle_years = middle_years.rename(columns={
    'n_on_track': 'n_met_expectations',
    'pct_on_track': 'pct_met_expectations',
    'n_mastered': 'n_exceeded_expectations',
    'pct_mastered': 'pct_exceeded_expectations',
    'subgroup': 'student_group',
    'pct_on_mastered': 'pct_met_exceeded'
})
middle_years.head()

,year,system,system_name,school,school_name,test,subject,grade,student_group,valid_tests,...,n_met_expectations,n_exceeded_expectations,pct_below,pct_approaching,pct_met_expectations,pct_exceeded_expectations,pct_met_exceeded,enrolled,tested,participation_rate
0,2018,10,Anderson County,2,Anderson County High School,EOC,Algebra I,10,All Students,43.0,...,**,**,**,**,**,**,**,NaN,NaN,NaN
1,2018,10,Anderson County,2,Anderson County High School,EOC,Algebra I,10,Economically Disadvantaged,14.0,...,**,**,**,**,**,**,7.1,NaN,NaN,NaN
2,2018,10,Anderson County,2,Anderson County High School,EOC,Algebra I,10,Non-Black/Hispanic/Native American,43.0,...,**,**,**,**,**,**,**,NaN,NaN,NaN
3,2018,10,Anderson County,2,Anderson County High School,EOC,Algebra I,10,Non-Economically Disadvantaged,29.0,...,**,**,**,**,**,**,**,NaN,NaN,NaN
4,2018,10,Anderson County,2,Anderson County High School,EOC,Algebra I,10,Non-English Learners/Transitional 1-4,43.0,...,**,**,**,**,**,**,**,NaN,NaN,NaN


### Combining `middle_years` dataset with 2022 to unify all data except 2017 

In [5]:
# Combine middle years with data_2022 
all_but_2017 = pd.concat([middle_years, data_2022])
all_but_2017.head(n=2)

,year,system,system_name,school,school_name,test,subject,grade,student_group,valid_tests,...,n_met_expectations,n_exceeded_expectations,pct_below,pct_approaching,pct_met_expectations,pct_exceeded_expectations,pct_met_exceeded,enrolled,tested,participation_rate
0,2018,10,Anderson County,2,Anderson County High School,EOC,Algebra I,10,All Students,43.0,...,**,**,**,**,**,**,**,NaN,NaN,NaN
1,2018,10,Anderson County,2,Anderson County High School,EOC,Algebra I,10,Economically Disadvantaged,14.0,...,**,**,**,**,**,**,7.1,NaN,NaN,NaN


In [19]:
# Calculate the number of unique systems and schools in each year
unique_counts = all_but_2017.groupby('year').agg({'system': pd.Series.nunique, 'school': pd.Series.nunique}).reset_index()

# Melt the DataFrame to get it in the right format for plotting
melted_counts = unique_counts.melt(id_vars='year', var_name='type', value_name='count')

# Create a bar chart with Plotly
fig = px.bar(melted_counts, x='year', y='count', color='type', barmode='group', title='Unique Counts of Systems and Schools per Year')

# Show the chart
fig.show()

### Fix Missing info in 2017

In [6]:
# Create a unique mapping of `all_but_2017` dataframe
mapping_all_but_2017 = all_but_2017.drop_duplicates(subset=['system', 'school'])

# Merge the dataframes
data_2017 = pd.merge(data_2017, mapping_all_but_2017[['system', 'system_name', 'school', 'school_name']], on=['system', 'school'], how='left')

data_2017.head(n=1)
# # Specify column order to match subsequent dataframes
# new_order = ['year', 'system', 'system_name', 'school', 'school_name', 'subject', 'grade', 'subgroup', 'valid_tests',
#              'n_below', 'n_approaching', 'n_on_track', 'n_mastered', 'pct_below', 
#              'pct_approaching', 'pct_on_track', 'pct_mastered', 'pct_on_mastered']

# # Reorder the columns
# data_2017 = data_2017[new_order]

,year,system,school,subject,grade,subgroup,valid_tests,n_below,n_approaching,n_on_track,n_mastered,pct_below,pct_approaching,pct_on_track,pct_mastered,pct_on_mastered,system_name,school_name
0,2017,10,2,Algebra I,10,All Students,50.0,**,**,**,**,**,**,**,**,**,Anderson County,Anderson County High School


looks like school `0` occurs as a missing value frequently.  Ill look at that

In [18]:
# Create new DF of missing 'system name' values
missing_2017 = data_2017[data_2017['system_name'].isna()]
missing_2017['system_name' == 10]

# # Drop all but system and school columns
# missing_2017 = missing_2017[['system', 'school']]

# # group by system and unique school
# missing_2017 = missing_2017.groupby('system')['school'].unique()
# missing_2017

KeyError: False

### Dataset info

In [9]:
data_2018.head(n=1)

,year,system,system_name,school,school_name,test,subject,grade,subgroup,valid_tests,n_below,n_approaching,n_on_track,n_mastered,pct_below,pct_approaching,pct_on_track,pct_mastered,pct_on_mastered
0,2018,10,Anderson County,2,Anderson County High School,EOC,Algebra I,10,All Students,43.0,**,**,**,**,**,**,**,**,**


In [10]:
data_2021.head(n=1)

,year,system,system_name,school,school_name,test,subject,grade,subgroup,participation_rate,...,valid_tests,n_below,n_approaching,n_on_track,n_mastered,pct_below,pct_approaching,pct_on_track,pct_mastered,pct_on_mastered
0,2021,10,Anderson County,2,Anderson County High School,EOC,Algebra I,10,All Students,96,...,50,**,**,**,**,**,**,**,**,**


In [11]:
# 